In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare,evaluate
from models.Encoders.FIM import FIM_Encoder
from models.Interactors import FIM_Interactor
from models.SFI import SFI_unified
from configs.ManualConfig import hparams

In [2]:
hparams['name'] = 'sfi'
hparams['select'] = 'unified'
hparams['k'] = 10
hparams['device'] = 'cuda:0'

In [3]:
vocab, loaders = prepare(hparams)

[2021-04-19 16:59:42,422] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'batch_size': 10, 'title_size': 20, 'abs_size': 40, 'his_size': 30, 'vert_num': 18, 'subvert_num': 293, 'npratio': 4, 'dropout_p': 0.2, 'query_dim': 200, 'embedding_dim': 300, 'filter_num': 150, 'value_dim': 16, 'head_num': 16, 'epochs': 8, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cuda:0', 'attrs': ['title'], 'k': 10, 'select': 'unified', 'save_step': [0], 'news_id': False, 'validate': False, 'interval': 10, 'spadam': True, 'onehot': False, 'name': 'sfi'}
[2021-04-19 16:59:42,425] INFO (root) preparing dataset...
[2021-04-19 16:59:46,311] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
record = next(iter(loaders[0]))

In [5]:
encoder = FIM_Encoder(hparams, vocab)
interactor = FIM_Interactor()
hparams['name'] = '-'.join([hparams['name'],hparams['select'],'fim',interactor.name])
sfi = SFI_unified(hparams, encoder, interactor).to(hparams['device'])

/home/peitian_zhang/anaconda3/envs/nn/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [6]:
sfi(record)

tensor([[-2.2131, -1.7967, -1.4670, -1.5024, -1.3035],
        [-1.5420, -1.4997, -1.5298, -1.9242, -1.6080],
        [-1.6073, -1.7635, -1.5281, -1.5721, -1.5917],
        [-1.7914, -1.6697, -1.4239, -1.6020, -1.5960],
        [-1.6786, -1.9920, -1.4564, -1.6708, -1.3635],
        [-1.7540, -1.2544, -1.6796, -1.7539, -1.7030],
        [-1.5683, -1.6042, -1.7248, -1.5517, -1.6071],
        [-1.3372, -1.6989, -2.1053, -1.6095, -1.4578],
        [-1.5293, -1.4227, -1.3967, -1.9654, -1.8660],
        [-1.2732, -2.0724, -1.5820, -1.4167, -1.9234]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)

In [7]:
sfi = train(sfi, hparams, loaders)

[2021-04-19 17:00:01,753] INFO (root) training...
epoch 1 , step 290 , loss: 1.5297: 100%|██████████| 295/295 [00:19<00:00, 15.25it/s]
[2021-04-19 17:00:21,969] INFO (root) saved model of step 0, epoch 1 at data/model_params/sfi-unified-fim-fim/demo_epoch1_step0_[hs=30,topk=10,attrs=title].model
epoch 2 , step 290 , loss: 1.4356: 100%|██████████| 295/295 [00:19<00:00, 15.30it/s]
[2021-04-19 17:00:42,121] INFO (root) saved model of step 0, epoch 2 at data/model_params/sfi-unified-fim-fim/demo_epoch2_step0_[hs=30,topk=10,attrs=title].model
epoch 3 , step 290 , loss: 1.3735: 100%|██████████| 295/295 [00:19<00:00, 15.13it/s]
[2021-04-19 17:01:02,504] INFO (root) saved model of step 0, epoch 3 at data/model_params/sfi-unified-fim-fim/demo_epoch3_step0_[hs=30,topk=10,attrs=title].model
epoch 4 , step 290 , loss: 1.2760: 100%|██████████| 295/295 [00:19<00:00, 15.34it/s]
[2021-04-19 17:01:22,665] INFO (root) saved model of step 0, epoch 4 at data/model_params/sfi-unified-fim-fim/demo_epoch4_st